# Attributi e descrittori

Iniziamo questa "seconda" parte sul linguaggio di programmazione Python.

In questi notebook analizzeremo dei concetto che ci porteranno ad una più approfondita comprensione degli aspetti del linguaggio che consentono di modificare e adattare il comportamento dell'interpete. Questo ci porterrà quindi ad una maggior comprensione dei meccanismi di metaprogramming di python, ma anche di altri linguaggi dinamici.

Il primo argomento di questa serie è il seguente notebook, ovvero andremo a comprendere il protocollo utilizzato dall'interprete per accedere agli attributi e il come sia possibile intervenire su di esso.

## Protocollo di accesso agli attributi

Dai primi notebook abbiamo imparato come accedere agli attributi con la nozione puntata.

In [9]:
class Padre:
    padre = "Sono il padre"

In [10]:
class Figlio(Padre):
    figlio = "Sono il figlio"

    def __init__(self, fratello = ""):
        self.fratello = fratello

In [11]:
figlio = Figlio("Sono il fratello")

print(figlio.padre)
print(figlio.figlio)
print(figlio.fratello)

Sono il padre
Sono il figlio
Sono il fratello


Questo meccanismo, che pare essere il classico meccanismo di altri linguaggi non dinamici come Java, in realtà nasconde al di sotto un processo altamente customizzabile, altamente dinamico.

Python utilizza tre metodi principali per l'accesso agli attributi:

- ```__getattribute__```, viene invocato per gli accessi in lettura
- ```__getattr__```, viene invocato da \__getattribute\__ qualora si verifichi una eccezzione, ovvero l'attributo non è presente
- ```__setattr__```, invocato per la scrittura di un attributo, se non è presente lo crea

Possiamo ripetere l'esempio soprastante con l'utilizzo di questi metodi.

In [12]:
figlio = Figlio("Sono il fratello")

print(figlio.__getattribute__('padre'))
print(figlio.__getattribute__('figlio'))
print(figlio.__getattribute__('fratello'))

Sono il padre
Sono il figlio
Sono il fratello


Dalle definizioni delle nostre classi è possibile capire che il metodo \__getattribute\__ chiamato dall'oggetto è definito nella classe ```object```. Infatti è possibile invocarlo direttamente dalla classe object.

In [14]:
figlio = Figlio("Sono il fratello")

print(object.__getattribute__(figlio, 'padre'))
print(object.__getattribute__(figlio, 'figlio'))
print(object.__getattribute__(figlio, 'fratello'))

Sono il padre
Sono il figlio
Sono il fratello


Nel caso in cui cercassimo di accedere ad un attributo che il nostro oggetto non contiene, allora l'interprete, dopo aver constatato che il \__getattribute\__ ha sollevato un eccezzione, prova a chiamare il metodo \__getattr\__. Attenzione, il metodo \__getattr\__ viene invocato solamente se è presente una definizione di esso nella classe, altrimenti passa l'eccezzione al chiamate.

Proviamo quindi ad implementare il metodo \__getattr\__ nella nostra classe.

In [22]:
class Padre:
    padre = "Sono il padre"

In [23]:
class Figlio(Padre):
    figlio = "Sono il figlio"

    def __init__(self, fratello = ""):
        self.fratello = fratello

    def __getattr__(self, attr):
        return f"Oggetti della classe Figlia non hanno l'attributo {attr}"

In [24]:
figlio = Figlio("Sono il fratello")

print(figlio.padre)
print(figlio.figlio)
print(figlio.fratello)
print(figlio.madre)

Sono il padre
Sono il figlio
Sono il fratello
Oggetti della classe Figlia non hanno l'attributo madre


Come possiamo notare dall'esempio, l'interprete invoca il metodo \__getattribute\__ sull'oggetto figlio per tentare di accedere all'attributo madre, ma questo attributo non è presente in figlio. Quindi il metodo \__getattribute\__, generando una eccezzione, invoca il metodo \__getattr\__ per gestire questo caso particolare. Il metodo \__getattr\__ è stato definito da noi e quindi ritornerà la stringa di testo per indicare che l'attributo non è presente.

Il metodo \__getattr\__ lo si può utilizzare per varie cose. Una di queste può essere il definire dinamicamente l'attributo che precedentemente non era presente. In questo modo sarà possibile accedervi in un secondo momento.

In [25]:
class Padre:
    padre = "Sono il padre"

In [26]:
class Figlio(Padre):
    figlio = "Sono il figlio"

    def __init__(self, fratello = ""):
        self.fratello = fratello

    def __getattr__(self, attr):
        print(f"Oggetti della classe Figlia non hanno l'attributo {attr}")
        self.__setattr__(attr, None)
        return None

In [27]:
figlio = Figlio("Sono il fratello")

print(figlio.padre)
print(figlio.figlio)
print(figlio.fratello)
print(figlio.madre)

Sono il padre
Sono il figlio
Sono il fratello
Oggetti della classe Figlia non hanno l'attributo madre
None


Questo processo è molto utile per la programmazione dinamica. Ma c'è da fare attenzione ad una cosa. Infatti se proviamo ad eseguire lo stesso codice, ma questa volta utilizzando \__getattribute\__ noteremo qualcosa di strano.

In [29]:
figlio = Figlio("Sono il fratello")

print(figlio.__getattribute__('padre'))
print(figlio.__getattribute__('figlio'))
print(figlio.__getattribute__('fratello'))
print(figlio.__getattribute__('madre'))

Sono il padre
Sono il figlio
Sono il fratello


AttributeError: 'Figlio' object has no attribute 'madre'

Questa volta ci da un errore. Perchè con l'utilizzo di \__getattribute\__ ci da errore e con la nozione puntata no?

La risposta è semplice. Perchè con la nozione puntata è l'interprete python che chiama il metodo \__getattribute\__ e quindi sarà ancora l'interprete a gestire l'eccezzione che genera il metodo stesso, non trovando l'attributo. Mentre se accediamo agli attributi tramite \__getattribute\__ saremo noi questa volta i chiamanti del metodo e quindi saremo ancora noi a dover gestire questa eccezzione.

Per gestire l'eccezzione possiamo banalmente utilizzare il try except come per qualsiasi altra eccezzione.

In [30]:
class Padre:
    padre = "Sono il padre"

    def __getattribute__(self, attr):
        print(f"Eseguito l'accesso all'attributo {attr}")
        try:
            return super().__getattribute__(attr)
        except AttributeError:
            return self.__getattr__(attr)

    def __getattr__(self, attr):
        print(f"Oggetti della classe Figlia non hanno l'attributo {attr}")
        self.__setattr__(attr, None)
        return None

In [31]:
class Figlio(Padre):
    figlio = "Sono il figlio"

    def __init__(self, fratello = ""):
        self.fratello = fratello

In [33]:
figlio = Figlio("Sono il fratello")

print(figlio.__getattribute__('padre'))
print(figlio.__getattribute__('figlio'))
print(figlio.__getattribute__('fratello'))
print(figlio.__getattribute__('madre'))

Eseguito l'accesso all'attributo __getattribute__
Eseguito l'accesso all'attributo padre
Sono il padre
Eseguito l'accesso all'attributo __getattribute__
Eseguito l'accesso all'attributo figlio
Sono il figlio
Eseguito l'accesso all'attributo __getattribute__
Eseguito l'accesso all'attributo fratello
Sono il fratello
Eseguito l'accesso all'attributo __getattribute__
Eseguito l'accesso all'attributo madre
Eseguito l'accesso all'attributo __getattr__
Oggetti della classe Figlia non hanno l'attributo madre
Eseguito l'accesso all'attributo __setattr__
None


In questo esempio abbiamo fatto due cose. La prima è di gestire l'eccezzione di \__getattribute\__, mentre la seconda è stata proprio definire il metodo \__getattribute\__.

L'abbiamo dovuto definire perchè altrimenti non sarebbe stato possibile gestire l'eccezzione di una chiamata derivante non dall'interprete. Quando facciamo l'override del metodo \__getattribute\__ bisogna fare attenzione a due cose:

- Non incorrere ad una ricorsione infinita
- Gestire l'eccezzioni derivanti da attributi non definiti

Per evitare una ricorsione infinita in \__getattribute\__ basta invocarlo tramite l'oggetto ```super()```. Facendo ciò andremo a chiedere alla superclasse di eseguire \__getattribute\__ e nel caso di eccezzione, la gestiamo chiamando \__getattr\__. Se al posto di super() avessimo utilizzato ```self```, allora si che avremmo una ricorsione infinita.

## Descrittori

I descrittori vengono utilizzati per controllare l'accesso agli attributi di altre classi. Sono quindi una generalizzazione delle property.

Tecnicamente un descrittore è una classe in cui è presente la definizione di almeno uno dei seguenti metodi magici:

- ```__get__(self, instance, cls)```
- ```__set__(self, instance, value)```
- ```__delete__(self, instance)```

Vediamo un primo esempio.

In [34]:
class descrittore:
    def __get__(self, instance, cls):
        print(instance)
        return 2

In [38]:
class Controllata:
    attr = descrittore()

In [41]:
c = Controllata()

print(c.attr)

2


Quando proviamo ad accedere all'attributo ```attr``` di ```c```, viene invocato il metodo ```__get__``` della classe ```descrittore``` dato che l'attributo attr è definito come istanza della classe descrittore.

In questo esempio non utilizziamo nessun parametro della classe descrittore. Però attraverso il parametro ```instance``` è possibile operare sulle singole istanze degli oggetti contenenti l'attributo descrittore.

Vediamo un secondo esempio.

In [44]:
class descrittore:
    def __get__(self, instance, cls):
        return instance.val + 1

In [45]:
class Controllata:
    attr = descrittore()

    def __init__(self, val):
        self.val = val

In [46]:
c1 = Controllata(2)
c2 = Controllata(23)

print(c1.attr)
print(c2.attr)

3
24


Infatti in questo esempio è evidente come il metodo \__get\__ della classe descrittore utilizzi l'attributo instance, che altro non è l'istanza di una classe, per recuperare valori dall'istanza stessa e ritornare valori a seconda di essi.

Il comportamento del metodo ```__set__``` a questo punto risulta evidente.

In [47]:
class descrittore:
    def __get__(self, instance, cls):
        return instance.val + 1

    def __set__(self, instance, value):
        instance.val = value

In [48]:
class Controllata:
    attr = descrittore()

    def __init__(self, val):
        self.val = val

In [49]:
c = Controllata(2)

print(c.attr)

c.attr = 32

print(c.attr)

3
33


Ma perchè il descrittore lo utilizziamo sempre sugli attributi di classe?

Il descrittore definisce un compotamento generale che si applica a tutte le istanze della classe istanza.

Se supponessimo di avere la seguente classe:

```python
class Controllata:
    def __init__(self):
        self.attr = descrittore()
```

In questo caso, ad ogni istanza della classe Controllata verrebbe associato un oggetto descrittore, creando una ridondanza inutile visto che il descrittore modella un unico comportamento che si applica ad ogni oggetto della classe controllata. Invece associando il descrittore all'attributo di classe viene istanziato una sola volta.

Concludiamo l'argomento dei destrittori e il notebook corrente con un esempio di uso realistico dei descrittori.

I file relativi a questo esempio li possiamo trovare dento ```codes/keyring```. In questo esempio realizziamo una sorta di keyring, cioè una strutturaa per memorizzaare informazioni protette da password.